In [ ]:
import os
import pandas as pd
import utm
from pyproj import CRS
import hickle as hkl
from scipy.ndimage import zoom
import rasterio
from rasterio.transform import from_origin
import geoarray as ga
import numpy as np
from arosics import COREG_LOCAL



In [ ]:
# import latlong point, use pyproj to find out crs
def get_crs(latitude, utminfo):
    if latitude>0:
        abool = False
    else: abool = True
    crs=CRS.from_dict({'proj':'utm', 'zone' : utminfo[2], 'south': abool})
    return crs

In [ ]:
def unsample_sentinel(data, idx, crs, utminfo):
    month=int(data.tile_acq_date[idx][:data.tile_acq_date[idx].find('/')]) # extract month from wv image acquisition date, so correct month of s2 can be selected
    s2_path = r"C:\Users\Gautam.Mathur\OneDrive - World Resources Institute\Gautam Intern Materials\sentinel2\chm-sentinel2"
    s2=hkl.load(f"{s2_path}\{data.plotid[idx]}.hkl")

    red = s2[month-1, :, :, 2] /65535
    green = s2[month-1, :, : ,1] / 65535
    blue = s2[month-1, :, :, 0] /65535
    newarrays = []
    for i in [red, green, blue]:
        img = zoom(i, zoom=4, order=3)
        newarrays.append(img)
    pixelsize = 280/newarrays[1].shape[0]
    top_left_easting = utminfo[0]-140
    top_left_northing = utminfo[1]+140
    transform = from_origin(top_left_easting, top_left_northing, pixelsize, pixelsize)
    s2name = 's2' + str(idx) + '.tif'
    s2raster = rasterio.open(s2name, 'w', driver='GTiff', height=newarrays[0].shape[0], width=newarrays[0].shape[1], dtype='float32', crs=crs, transform=transform, count=3)
    s2raster.write(newarrays[0], 1)
    s2raster.write(newarrays[1], 2)
    s2raster.write(newarrays[2], 3)
    s2raster.close()

In [ ]:
def resample_wv(data, idx, crs, utminfo):
    worldview_path = r'C:\Users\Gautam.Mathur\OneDrive - World Resources Institute\Gautam Intern Materials\worldview-labels-heightpreds\chm-preds'
    maxar = rasterio.open(f"{worldview_path}\{data.filename[idx]}").read().astype(np.float32) / 255
    maxarrays = [maxar[0], maxar[1], maxar[2]]
    maxar5 = []
    for i in maxarrays:
        img = zoom(i, zoom=1/5, order=3)
        maxar5.append(img)
    pixelsize = 0.59*maxarrays[1].shape[0]/maxar5[1].shape[0]
    top_left_easting = utminfo[0]-70
    top_left_northing = utminfo[1]+70
    transform = from_origin(top_left_easting, top_left_northing, pixelsize, pixelsize)
    wvname = "wv" + str(idx) + ".tif"
    wvraster = rasterio.open(wvname, 'w', driver='GTiff', height=maxar5[0].shape[0], 
                         width=maxar5[0].shape[1], dtype='float32', crs=crs, transform=transform, count=3)
    wvraster.write(maxar5[0], 1)
    wvraster.write(maxar5[1], 2)
    wvraster.write(maxar5[2], 3)
    wvraster.close()
#0.59 m GSD

In [ ]:
def coreg(idx, folder):
    im_reference = 'wv' + str(idx) + ".tif"
    im_target = 's2' + str(idx) + ".tif"
    outpath = os.path.join(folder, 'coreg'+str(idx)+".bsq")
    kwargs = {
        'grid_res' : 2, 
        'window_size' : (18, 18),
        'path_out' : outpath,
        'projectDir' : folder,
        'max_shift' : 10, #this is in number of pixels. I wrote 10 because resampled spatial resolution of wv ~ 2.95. There is a default max shift of 5 pixels, so we are expanding max shift in this case. 
        'q' : False,}
    CRL = COREG_LOCAL(im_reference,im_target,**kwargs, min_reliability=0)
    CRL.correct_shifts()

In [ ]:
def resample_result(crs, idx, utminfo):
    path = 'coreg'+str(idx)+".bsq"
    bsq = rasterio.open(path).read().astype(np.float32)
    shiftarrays = []
    for i in [bsq[0], bsq[1], bsq[2]]:
        img=zoom(i, zoom=1/4, order=3)
        shiftarrays.append(img)
    pixelsize = 10
    top_left_easting = utminfo[0]-140
    top_left_northing = utminfo[1]+140
    from rasterio.transform import from_origin
    transform = from_origin(top_left_easting, top_left_northing, pixelsize, pixelsize)
    finalpath = os.path.join("resampled_results", str(idx)+"final.tif")
    cr110m = rasterio.open(finalpath, 'w', driver='GTiff', height=shiftarrays[0].shape[0], 
                            width=shiftarrays[0].shape[1], dtype='float32', crs=crs, transform=transform, count=3)
    cr110m.write(shiftarrays[0], 1)
    cr110m.write(shiftarrays[1], 2)
    cr110m.write(shiftarrays[2], 3)
    cr110m.close()


In [ ]:
os.chdir(r'C:\Users\Gautam.Mathur\All_Images')
#os.mkdir("resampled_results")
data = data = pd.read_csv(r"C:\Users\Gautam.Mathur\OneDrive - World Resources Institute\Gautam Intern Materials\chm-master-clean.csv")
folder = r'C:\Users\Gautam.Mathur\All_Images'
for idx in range(0, len(data.lon)):
    latitude = data.lat[idx]
    longitude = data.lon[idx]
    utminfo = utm.from_latlon(latitude, longitude)
    crs = get_crs(latitude, utminfo)
    unsample_sentinel(data, idx, crs, utminfo)
    resample_wv(data, idx, crs, utminfo)
    coreg(idx, folder)
    resample_result(crs, idx, utminfo)